In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoModelForCausalLM, AutoTokenizer, pipeline

## Translate

- https://huggingface.co/google/madlad400-3b-mt
- https://github.com/google-research/t5x

In [3]:
# model_name = 'jbochi/madlad400-3b-mt'
# model = T5ForConditionalGeneration.from_pretrained(model_name, device_map="cpu") #
# # model = T5ForConditionalGeneration.from_pretrained(model_name, device_map="auto")
# tokenizer = T5Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/11.8G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/830 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.43M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.6M [00:00<?, ?B/s]

In [4]:
# def translate(text, lang): # lang = ru, en
#   text = f"<2{lang}> I love pizza!"
#   input_ids = tokenizer(text, return_tensors="pt").input_ids.to(model.device)
#   outputs = model.generate(input_ids=input_ids)

#   return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [6]:
%%time
translate('I love pizza!', 'ru') # on cpu

CPU times: user 6.21 s, sys: 2.02 s, total: 8.23 s
Wall time: 28.3 s


'Я люблю пиццу!'

In [2]:
pipe_to_ru = pipeline("translation", model="Helsinki-NLP/opus-mt-en-ru")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda:0


In [3]:
pipe_to_en = pipeline("translation", model="Helsinki-NLP/opus-mt-ru-en")

Device set to use cuda:0


In [16]:
%%time
pipe_to_ru('I love pizza!')

CPU times: user 347 ms, sys: 225 ms, total: 572 ms
Wall time: 1.46 s


[{'translation_text': 'Я люблю пиццу!'}]

In [19]:
%%time
pipe_to_en('Я люблю пиццу!')

CPU times: user 93.3 ms, sys: 0 ns, total: 93.3 ms
Wall time: 93.5 ms


[{'translation_text': 'I love pizza!'}]

## Base model

https://huggingface.co/microsoft/Phi-3-mini-4k-instruct

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

# Create a pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=500,
    do_sample=False,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


In [90]:
def pred(msg, temperature=None, top_p=None):
  messages = []
  for i in msg:
    tmp = dict()
    for key, value in i.items():
      if key != 'role':
        tmp[key] = pipe_to_en(value)[0]['translation_text']
      else:
        tmp[key] = value
    messages.append(tmp)
  print(messages)

  if not temperature and not top_p:
    output = pipe(messages)
  elif temperature and not top_p:
    output = pipe(messages, temperature=temperature)
  elif not temperature and top_p:
    output = pipe(messages, top_p=top_p)
  else:
    output = pipe(messages, temperature=temperature, top_p=top_p)



  result = pipe_to_ru(output[0]['generated_text'])
  return result[0]['translation_text']


In [19]:
messages = [
    {"role": "user", "content": "Создай рассказ об x"},
    {"role": "system", "content": "Create 123"}
]
pred(messages)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


[{'role': 'user', 'content': 'Create a story about x'}, {'role': 'system', 'content': 'Create 123'}]


'Когда-то, в далёкой стране, жила молодая девочка по имени Лили. Она была известна по всей деревне своим добрым сердцем и любовью к приключениям. Однажды, изучая густый лес возле своего дома, Лили наткнулась на загадочный, светящийся камень. Камень, бульдозером которого была другая мировая энергия, и как только она прикоснулась к нему, она была доставлена в магическое пространство, полное мифических созданий и зачарованных лесов. В этом новом мире, Лили встретила мудрого старого волшебника по имени Мерлин, который объяснил, что этот камень является мощным артефактом, известным как «Х», который имел возможность дать наследнику одно единственное желание. Однако использование Х было сопряжено с большой ответственностью, так как оно также могло принести большую опасность и хаос, если оно было беззаботно использовано. Лили, желая помочь тем, кто в нем нуждался, она решила использовать X для того, чтобы принести людям волшебного мира благополучие и счастье. Она пожелала положить конец страда

In [15]:
len('Когда-то, в далёкой стране, жила молодая девочка по имени Лили. Она была известна по всей деревне своим добрым сердцем и любовью к приключениям. Однажды, изучая густый лес возле своего дома, Лили наткнулась на загадочный, светящийся камень. Камень, бульдозером которого была другая мировая энергия, и как только она прикоснулась к нему, она была доставлена в магическое пространство, полное мифических созданий и зачарованных лесов. В этом новом мире, Лили встретила мудрого старого волшебника по имени Мерлин, который объяснил, что этот камень является мощным артефактом, известным как «Х», который имел возможность дать наследнику одно единственное желание. Однако использование Х было сопряжено с большой ответственностью, так как оно также могло принести большую опасность и хаос, если оно было беззаботно использовано. Лили, желая помочь тем, кто в нем нуждался, она решила использовать X для того, чтобы принести людям волшебного мира благополучие и счастье. Она пожелала положить конец страданиям, причиня ужасную засуху, которая обрушилась на земле годами. В тот момент, открылось лы, и проли дожде, и шел дождь, ради жизни и пасть на земле.')

1149

## Задание

## 1. Zero-Shot Инструкция

**Цель**: продемонстрировать, что даже без примеров (zero-shot) модель может понять задачу, если она описана достаточно чётко.

**Задание**:
1. Выберите любую модель (например, `google/flan-t5-small`) и настройте пайплайн генерации текста.
2. Сформулируйте короткий промт (1–2 предложения), который просит модель:
   - дать определение термина (например, «блокчейн», «иммунизирующее вещество» и т.д.) простым языком;
   - ограничить размер ответа (например, «не более 40 слов»).
3. Наблюдайте результат. Затем измените промт так, чтобы ответ стал более конкретным (попросите привести пример).
4. Сравните ответы — опишите, что изменилось после уточнения промта.

**Критерии успеха**:
- Модель выдаёт осмысленный ответ.
- Студент чётко описал, чем отличается первый ответ (zero-shot) от уточнённого.

In [34]:
messages = [
    {"role": "user", "content": '''Роль: эксперт в биологии. Инструкция: раскрой значения термина фотосинтез.
    Контекст: изложи данное понятие доступно и кратко в 40 слов
    .'''},
]
pred(messages)

[{'role': 'user', 'content': 'Role: Expert in biology: Instruction: Open up the meaning of photosynthesis. Context: state this concept in 40 words.'}]


'Фотосинтез представляет собой процесс, в рамках которого зеленые растения, водоросли и некоторые бактерии преобразуют световую энергию, обычно от солнца, в химическую энергию в виде глюкозы, используя воду и углекислый газ и выделяя кислород в качестве побочного продукта.'

In [29]:
messages = [
    {"role": "user", "content": '''Роль: эксперт в биологии. Инструкция: раскрой значения термина фотосинтез.
    Контекст: изложи данное понятие доступно и кратко в 40 слов
    Формат: сначала привели определение, а потом пример'''},
]
pred(messages)

[{'role': 'user', 'content': 'Role: An expert in biology: Instruction: Open up the meaning of photosynthesis. Context: state the term in 40 words The Format: First gave a definition and then an example.'}]


'Фотосинтез — это процесс, в рамках которого зеленые растения, водоросли и некоторые бактерии преобразуют световую энергию в химическую энергию, сохраняя ее в глюкозе. Пример: подсолнух использует фотосинтез для преобразования солнечного света в энергию, подпитывая ее рост.'

**Вывод:**
С помощью добаления инструкции format можно направить нейросеть на раскрытие определенных аспектов заданной области.


## 2. Приём «Роль и стиль» (Role + Style Prompt)

**Цель**: научиться управлять «тоном» и «ролью» модели, чтобы ответ был в нужном стиле.

**Задание**:
1. Сформулируйте промт: «Ты — [роль], напиши текст в стиле [указать стиль]…».
   - Пример роли: «редактор научных журналов», «литературный критик», «юмористический блогер» и т.д.
   - Пример стиля: «формальный», «дружелюбный», «саркастический» и т.д.
2. Предложите модели сгенерировать небольшой текст (5–7 предложений) на заданную тему (например, обзор нового фильма, описание учебного курса).
3. Сравните тексты для разных ролей/стилей. Как меняется лексика, тон, формулировки?

**Критерии успеха**:
- ПОказано минимум два разных стиля (два запуска модели) и указаны отличия.
- Выполнен краткий анализ, насколько модель следует заданной роли.

In [40]:
messages = [
    {"role": "user", "content": '''Ты на - шеф-повар, напиши текст в художественном стиле о приготовлении пиццы.Составь от 5 до 7 предложений, раскрывающих суть темы
  '''},
]
pred(messages)

[{'role': 'user', 'content': "You're a chef, write an artistic text about making pizza. Make between 5 and 7 sentences to explain the subject."}]


'В качестве шеф-повара я нахожу искусство пиццы, делающей из себя восхитительный танец вкусов и текстуры. Процесс начинается с создания шелковистого, эластичного теста, кнута с любовью и заботой. Затем тест мягко растягивается и формируется, формируя идеальный холст для энергичных напитков. Распространяется щедрый слой богатого томатного соуса, за которым следует симфония свежих ингредиентов, от суккуленного мяса до ароматических трав. Пицца затем выпекается до совершенства, корка становится золотой и хрустящей, в то время как сыр плавится в гуи, неотразимое одеяло. Последний шедевр - это гармоничное сочетание ароматов, текстуров и цветов, истинное произведение искусства, которое радует чувства и согревает душу.'

In [41]:
messages = [
    {"role": "user", "content": '''Ты - шеф-повар, напиши текст в научном стиле о приготовлении пиццы.Составь от 5 до 7 предложений, раскрывающих суть темы
  '''},
]
pred(messages)

[{'role': 'user', 'content': "You're a chef, write a scientific text about making pizza. Make between 5 and 7 sentences to explain the subject."}]


'Процесс производства пиццы включает в себя ряд научных принципов и методов, способствующих его вкусу. Во-первых, тесто готовится путем сочетания муки, воды, дрожжей и соли, которые подвергаются ферментации, биологического процесса, в котором дрожжи потребляют сахар и производят газ диоксида углерода, что приводит к росту теста. Затем тесто скручивается для развития глютена, белковой сети, которая даёт корке пиццы ее жвачную текстуру. После отдыха тест выкачивается и накачивается томатным соусом, сыром и различными ингредиентами, которые затем печутся в горячей печи. Мейльардская реакция, химическая реакция между аминокислотами и снижением сахара, происходит во время выпекания, создавая золотую корку и сложный ароматический профиль. Высокая теплота также заставляет сыр плавиться, а соус густится, создает вкусную и удовлетворительную пиццу. В целом наука о пицце включает сочетание биологических, химических и физических процессов, которые приводят к восхитному и приятному блюдению.'

In [43]:
messages = [
    {"role": "user", "content": '''Ты - профессор по химии, напиши текст в художественном стиле о приготовлении пиццы.Составь от 5 до 7 предложений, раскрывающих суть темы
  '''},
]
pred(messages)

[{'role': 'user', 'content': "You're a chemistry professor, write an art-style text about pizza making. Make between 5 and 7 sentences explaining the subject."}]


'В алхимическом танце пиццы, тест — это холст, и ингредиенты — это пигменты, которые приносят шедевр. Мука, вода, дрожжи и соль — это первичные цвета, смешивающие и преобразующие под нежным прикосновением рук пекарня. По мере увеличения теста он вдыхает жизнь в творение, расширение и размягчение, как и щетки художника на пустом холсте. Соус, живой красный — это первый слой цвета, богатый ароматами томатов, трав и специй. Сыр, золотой хватка, плавка и сладости соусом, создающий связную текстуру и глубину. Потрошения, калейдоскоп цветов и ароматов — это последние прикосновения, каждый из которых добавляет свой уникальный характер к составу. И когда пицца выпекается, тепло преобразует ингредиенты, создает симфонию ароматов и текстуры, которые одновременно являются праздником для чувств и произведения искусства.'

In [42]:
messages = [
    {"role": "user", "content": '''Ты - профессор по химии, напиши текст в научном стиле о приготовлении пиццы.Составь от 5 до 7 предложений, раскрывающих суть темы
  '''},
]
pred(messages)

[{'role': 'user', 'content': "You're a chemistry professor, write a scientific text about making pizza. Make between 5 and 7 sentences explaining the subject."}]


'Производство пиццы - это кулинарный процесс, который включает в себя серию химических реакций и трансформаций. Душ, являющийся основой пиццы, состоит из комбинирования муки, воды, дрожжей и соли. Дрожжи, живой организм, подвергаются ферментации, вырабатывают газ диоксида углерода, который поднимается. Этот процесс, известный как осадок, приводит к легкой и воздушной текстуре. Соус, обычно производимый из помидоров, проходит процесс, называемый сокращением, когда содержание воды испаряется, концентрируется ароматы и густой соус. Сыр, обычно моззарелла, растает при нагревании, из-за белков и жиров в сыре, который претерпевает фазу изменения из твердого в жидкость. Этот процесс плавки, известный как уплотнение, создает гусиный и восхитительный процесс. Этот процесс смешан с гладким продуктом, испаривается с помощью гладких процессов. Этот процесс химости вызывает мая реакция, химическая реакция между аминокислотами и снижением сахара, который дает пице свой характер коровой корону и бога

**Вывод:**
При изменении стиля и роли модель пытаестся им следовать.

В первых 2-х вариантах модель рассказывала о процессе с точки зрения шеф-повара, и описывала процесс со стороны близкой к пошаговой инструкции

При изменении роли на 'профессор по химии' модель стала описывать процесс с точки зрения химических процессов, которые поисходят при приготовлении

## 3. Few-Shot Пример (Обучение на примерах)

**Цель**: показать, как несколько примеров в промте улучшают ответы модели на специфичную задачу.

**Задание**:
1. Выберите задачу классификации тональности (положительная / нейтральная / отрицательная) или любую другую простую задачу (например, распознавание простых категорий).
2. Составьте промт в стиле Few-Shot:
   - Дайте 2–3 примера (вход → ответ).
   - Затем попросите модель классифицировать новое предложение.
3. Для проверки запустите:
   - **Вариант A**: без примеров (zero-shot).
   - **Вариант B**: с 2–3 примерами (few-shot).
4. Сравните результаты.

**Критерии успеха**:
- Продемонстрировать, что при наличии примеров (few-shot) точность и релевантность ответа выше.
- Описать, насколько изменилось качество вывода.

In [66]:
messages = [
    {"role": "user", "content": '''Задача: классифицировать тексты по тональности.
    Формат: название тональности
     Контекст: ЭТО ПОЗОР! Человек занимающий подобную должность обязан знать Английский!!!
    .'''},
]
pred(messages)

[{'role': 'user', 'content': 'Objective: To classify the text by tone, the format: the tonality of the Context: This POSOR! The person holding such a position must know English!'}]


'Тональность представленного контекста является сарказмом. Использование "POSOR!" (аббревиатура для "Пожалуйста, откройте кое-какую другую комнату") и преувеличенное заявление о том, что человек должен знать английский язык, чтобы занять такую позицию, свидетельствуют о сарказме. Оратор высмеивает идею о том, что знание английского является требованием для этой роли, что означает, что это абсурдное или завышенное ожидание.'

In [65]:
messages = [
    {"role": "user", "content": '''Задача: классифицировать тексты по тональности.
    Формат: название тональности
    Контекст: ЭТО ПОЗОР! Человек занимающий подобную должность обязан знать Английский!!!
    .'''}, # : Положительная, Отрицательная, Нейтральная.
    {"role": "assistant", "content": "Отрицательная"},
     {"role": "user", "content": 'Контекст: мозги, конечно не видно'},
    {"role": "assistant", "content": "Нейтральная"},
    {"role": "user", "content": 'Контекст: а вот что такое ,,барокамера,,-я не знаю.....((( '}
]
pred(messages)

[{'role': 'user', 'content': 'Objective: To classify the text by tone, the format: the tonality of the Context: This POSOR! The person holding such a position must know English!'}, {'role': 'assistant', 'content': 'Negative'}, {'role': 'user', 'content': "Context: brains, of course, can't be seen."}, {'role': 'assistant', 'content': 'Neutral'}, {'role': 'user', 'content': "Context: this is what a baroquer is, I don't know."}]


'Нейтральный'

**Вывод:**
При предоставлении модели примеров, она перестала рассуждать о предложении и выдала релевантный ответ. Примеры помогли модели с пониманием задачи и формарованием корректного ответа.

## 4. Сложный контекст (Long Context Prompt) и Вопрос-Ответ

**Цель**: научиться выдавать модели длинный контекст (абзац/статью) и формулировать вопросы, проверяя, как модель «извлекает» ответ.

**Задание**:
1. Найдите (или составьте) короткий текст (3–5 предложений), описывающий событие или факт (например, «В 2022 году в городе N…»).
2. В промте передайте этот текст и задайте 2–3 вопроса по содержанию (например, «В каком году…?», «Как зовут…?»).
3. Убедитесь, что ответ на вопрос явно есть в тексте.
4. Проверьте, может ли модель точно найти ответ.
5. Дополнительно попробуйте задать вопрос, ответ на который **не** содержится в тексте, и проанализируйте, что делает модель (признаки «галлюцинации»).

**Критерии успеха**:
- Модель корректно отвечает на вопросы, ответ на которые содержится в тексте.
- Вы демонстрируете понимание, почему при «неизвестном» вопросе модель может «выдумывать».

In [69]:
messages = [
    {"role": "system", "content": '''Задача: ответить на вопросы по тексту. Текст:
     15 февраля 2013 года метеороид диаметром около 18 метров и массой порядка 11 тыс. тонн (по расчётам НАСА) вошёл в атмосферу
      Земли на скорости около 18,6 км/с. Судя по продолжительности атмосферного полёта, вход в атмосферу произошёл под очень острым
      углом. Спустя примерно 32,5 сек после этого небесное тело разрушилось над Челябинской областью. Разрушение представляло
      собой серию событий, сопровождавшихся распространением ударных волн. Общее количество высвободившейся энергии по
       оценкам НАСА составило около 440 килотонн в тротиловом эквиваленте
    .'''},
    {"role": "user", "content": 'Где произошло падение метеорита?'},
]
pred(messages)

[{'role': 'system', 'content': 'Objective: To answer questions on the text: On 15 February 2013, a meteoroid of about 18 metres in diameter and a mass of about 11,000 tons (as calculated by NASA) entered the Earth &apos; s atmosphere at a speed of about 18.6 km/s. Based on the length of the atmospheric flight, the entrance to the atmosphere occurred at a very sharp angle. Some 32.5 seconds later, the celestial body collapsed over the Chelyabinsk region. The destruction was a series of events accompanied by the spread of shock waves. NASA estimated the total amount of energy released was about 440 kilotons in TNT equivalent.'}, {'role': 'user', 'content': 'Where did the meteorite fall?'}]


'Метеорит упал над Челябинской области.'

In [71]:
messages = [
    {"role": "system", "content": '''Задача: ответить на вопросы по тексту. Текст:
     15 февраля 2013 года метеороид диаметром около 18 метров и массой порядка 11 тыс. тонн (по расчётам НАСА) вошёл в атмосферу
      Земли на скорости около 18,6 км/с. Судя по продолжительности атмосферного полёта, вход в атмосферу произошёл под очень острым
      углом. Спустя примерно 32,5 сек после этого небесное тело разрушилось над Челябинской областью. Разрушение представляло
      собой серию событий, сопровождавшихся распространением ударных волн. Общее количество высвободившейся энергии по
       оценкам НАСА составило около 440 килотонн в тротиловом эквиваленте
    .'''},
    {"role": "user", "content": 'В каком году произошло данное событие?'}
]
pred(messages)

[{'role': 'system', 'content': 'Objective: To answer questions on the text: On 15 February 2013, a meteoroid of about 18 metres in diameter and a mass of about 11,000 tons (as calculated by NASA) entered the Earth &apos; s atmosphere at a speed of about 18.6 km/s. Based on the length of the atmospheric flight, the entrance to the atmosphere occurred at a very sharp angle. Some 32.5 seconds later, the celestial body collapsed over the Chelyabinsk region. The destruction was a series of events accompanied by the spread of shock waves. NASA estimated the total amount of energy released was about 440 kilotons in TNT equivalent.'}, {'role': 'user', 'content': 'What year did this event take place?'}]


'Это мероприятие состоялось в 2013 году.'

In [73]:
messages = [
    {"role": "system", "content": '''Задача: ответить на вопросы по тексту. Текст:
     15 февраля 2013 года метеороид диаметром около 18 метров и массой порядка 11 тыс. тонн (по расчётам НАСА) вошёл в атмосферу
      Земли на скорости около 18,6 км/с. Судя по продолжительности атмосферного полёта, вход в атмосферу произошёл под очень острым
      углом. Спустя примерно 32,5 сек после этого небесное тело разрушилось над Челябинской областью. Разрушение представляло
      собой серию событий, сопровождавшихся распространением ударных волн. Общее количество высвободившейся энергии по
       оценкам НАСА составило около 440 килотонн в тротиловом эквиваленте
    .'''},
    {"role": "user", "content": 'Были ли разрушения?'}
]
pred(messages)

[{'role': 'system', 'content': 'Objective: To answer questions on the text: On 15 February 2013, a meteoroid of about 18 metres in diameter and a mass of about 11,000 tons (as calculated by NASA) entered the Earth &apos; s atmosphere at a speed of about 18.6 km/s. Based on the length of the atmospheric flight, the entrance to the atmosphere occurred at a very sharp angle. Some 32.5 seconds later, the celestial body collapsed over the Chelyabinsk region. The destruction was a series of events accompanied by the spread of shock waves. NASA estimated the total amount of energy released was about 440 kilotons in TNT equivalent.'}, {'role': 'user', 'content': 'Was there destruction?'}]


'Да, это было разрушение. Метеороид вошел в атмосферу Земли под очень острым углом, что привело к ее распаду и высвобождению значительного количества энергии. Выпущенная энергия была эквивалентна примерно 440 килотоннам ТНТ. Это привело к серии событий, включая распространение ударных волн, которые причинили ущерб и разрушения в Челябинской области.'

In [72]:
messages = [
    {"role": "system", "content": '''Задача: ответить на вопросы по тексту. Текст:
     15 февраля 2013 года метеороид диаметром около 18 метров и массой порядка 11 тыс. тонн (по расчётам НАСА) вошёл в атмосферу
      Земли на скорости около 18,6 км/с. Судя по продолжительности атмосферного полёта, вход в атмосферу произошёл под очень острым
      углом. Спустя примерно 32,5 сек после этого небесное тело разрушилось над Челябинской областью. Разрушение представляло
      собой серию событий, сопровождавшихся распространением ударных волн. Общее количество высвободившейся энергии по
       оценкам НАСА составило около 440 килотонн в тротиловом эквиваленте
    .'''},
    {"role": "user", "content": 'После падения метеорита, куда его поместили на хранение?'}
]
pred(messages)

[{'role': 'system', 'content': 'Objective: To answer questions on the text: On 15 February 2013, a meteoroid of about 18 metres in diameter and a mass of about 11,000 tons (as calculated by NASA) entered the Earth &apos; s atmosphere at a speed of about 18.6 km/s. Based on the length of the atmospheric flight, the entrance to the atmosphere occurred at a very sharp angle. Some 32.5 seconds later, the celestial body collapsed over the Chelyabinsk region. The destruction was a series of events accompanied by the spread of shock waves. NASA estimated the total amount of energy released was about 440 kilotons in TNT equivalent.'}, {'role': 'user', 'content': 'After the meteorite fell, where was it stored?'}]


'Извините, но я не могу предоставить информацию в реальном масштабе времени или в конкретном месте о фрагментах метеорита Челябинского события. После такого события осколки метеорита обычно собираются учеными и исследователями для изучения. Точное место, где хранятся фрагменты, зависит от научно-исследовательских учреждений или организаций, занимающихся изучением метеорита. В случае хельябинского метеорита осколки были собраны местными властями и переданы Российской академии наук. Институт земной креста (Иркутскский филиал) и Институт Земли и Краста (Московский филиал) отвечали за анализ и хранение осколков метеорита. Осколки метеорита хранились в безопасных помещениях этих учреждений, где они были изучены и изучены учеными, чтобы узнать больше о составе, происхождении и истории метеорита. Результаты исследований могут помочь нам лучше понять метеоритов и их воздействие на Землю.'

In [74]:
messages = [
    {"role": "system", "content": '''Задача: ответить на вопросы по тексту. Текст:
     15 февраля 2013 года метеороид диаметром около 18 метров и массой порядка 11 тыс. тонн (по расчётам НАСА) вошёл в атмосферу
      Земли на скорости около 18,6 км/с. Судя по продолжительности атмосферного полёта, вход в атмосферу произошёл под очень острым
      углом. Спустя примерно 32,5 сек после этого небесное тело разрушилось над Челябинской областью. Разрушение представляло
      собой серию событий, сопровождавшихся распространением ударных волн. Общее количество высвободившейся энергии по
       оценкам НАСА составило около 440 килотонн в тротиловом эквиваленте
    .'''},
    {"role": "user", "content": 'Исходя из приведеных фактов какой характер имел Жилин?'}
]
pred(messages)

[{'role': 'system', 'content': 'Objective: To answer questions on the text: On 15 February 2013, a meteoroid of about 18 metres in diameter and a mass of about 11,000 tons (as calculated by NASA) entered the Earth &apos; s atmosphere at a speed of about 18.6 km/s. Based on the length of the atmospheric flight, the entrance to the atmosphere occurred at a very sharp angle. Some 32.5 seconds later, the celestial body collapsed over the Chelyabinsk region. The destruction was a series of events accompanied by the spread of shock waves. NASA estimated the total amount of energy released was about 440 kilotons in TNT equivalent.'}, {'role': 'user', 'content': 'On the basis of the facts, what was the nature of Gilin?'}]


'В представленном тексте ничего не говорится о "Гилине". В вопросе может возникнуть недопонимание или опечатка. Информация о метеорите, который вошел в атмосферу Земли 15 февраля 2013 года, и о последующих событиях, которые произошли в результате его включения. Если вы имеете в виду другую тему или если вам необходима информация по конкретной теме, просьба указать дополнительный контекст или разъяснить ваш вопрос.'

**Вывод:**
Модель старается отвечать по приведенному контексту, но при некоторых вопросах при ответе она выходит за рамки приведенного текста, данное поведение можно объяснить тем, что в обучающей выборке был текст связанный с метеоритами из которого нейросеть попыталась достроить ответ.

Выбранная модель обучена на специальных вопросах вне контекста, на которые не нужно приводить ответ.

Так как в основе модели лежат вероятности, то на любой вопрос модель может рассчитать данные вероятности и получить ответ.

## 5. Итеративное уточнение (Iterative Refinement)

**Цель**: показать, что можно улучшать ответ модели, постепенно корректируя промт.

**Задание**:
1. Сформулируйте начальный промт: «Объясни, почему важно изучать математику в школе» (пример).
2. После первого ответа попросите модель «Сократить текст до 2 предложений и сделать его более формальным» (или любая другая корректировка).
3. Затем попросите модель «Добавить 1–2 примера практического применения».
4. Оцените финальную версию ответа: действительно ли модель учитывает все изменения?

**Критерии успеха**:
- продемонстрирована цепочка минимум из трёх шагов и покажет, как каждый следующий промт уточняет ответ.

In [75]:
messages = [
    {"role": "user", "content": 'Объясни почему важно не допускать утечек памяти в C++?'}
]
pred(messages)

[{'role': 'user', 'content': 'Explain why it is important to prevent C++ memory leaks?'}]


Your input_length: 477 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


'Недопущение утечек памяти в C++ имеет решающее значение по нескольким причинам, каждый из которых способствует общему здоровью, эффективности и надежности программного обеспечения. Утечка памяти происходит, когда программа распределяет возможную память динамическо (с помощью "нового", "маллока" и т.д.), но не позволяет вернуть ее обратно в систему после того, как в ней больше нет необходимости. Это может привести к нескольким проблемам: 1. ** Снижение эффективности**: со временем утечка памяти может существенно ухудшить функционирование приложения. Это означает, что в случае утечки в памяти может возникнуть дополнительная утечка, имеющаяся в системе память уменьшается, что может замедлить применение и даже привести к авариям, если система выключится. ** Испарение памяти: в течение длительного времени утечка памяти может привести к истощению ресурсов. Это приложение может поглотить всю имеющуюся память, что ведет к нестабильности системы или катастрофам. Это особенно проблематично в ус

In [77]:
messages = [
    {"role": "user", "content": '''Объясни почему важно не допускать утечек памяти в C++?
    Формат: 2 предложения с наиболее важной информацией
    .'''}
]
pred(messages)

[{'role': 'user', 'content': 'Explain why it is important to prevent C++ memory leaks? Format: 2 sentences with the most important information.'}]


'Предотвращение утечек памяти C++ имеет решающее значение, поскольку они могут привести к более широкому использованию памяти, что может привести к тому, что программа закроется и разрушится. Кроме того, утечка памяти может со временем ухудшить функционирование системы, что приведет к замедлению реакции приложения и увеличению потребления ресурсов.'

In [78]:
messages = [
    {"role": "user", "content": '''Объясни почему важно не допускать утечек памяти в C++?
    Формат: 2 предложения с наиболее важной информацией
    Контекст: приведи 1 или 2 примера критических ошибок, связанных с данной проблемой
    .'''}
]
pred(messages)

[{'role': 'user', 'content': 'Explain why it is important to prevent C++ memory leaks? Format: 2 sentences with the most important information Context: give 1 or 2 examples of critical errors associated with the problem.'}]


'Предотвращение утечек памяти C++ имеет решающее значение, поскольку они могут привести к более широкому использованию памяти, что может привести к тому, что программа закроется и разоблачится. Например, в вебсервере утечка памяти может со временем ухудшить производительность, что может привести к замедлению времени реагирования или даже к неисправности системы.'

In [79]:
messages = [
    {"role": "user", "content": '''Объясни почему важно не допускать утечек памяти в C++?
    Формат: 2 предложения с наиболее важной информацией
    Контекст: приведи 1 или 2 примера критических ошибок, связанных с данной проблемой.
     и приведи рекоммендации, по профилактике данной проблемы
    .'''}
]
pred(messages)

[{'role': 'user', 'content': 'Explain why it is important to prevent C++ memory leaks? Format: 2 sentences with the most important information Context: give 1 or 2 examples of critical errors associated with the problem. and provide recommendations to prevent the problem.'}]


'Предотвращение утечек памяти C++ имеет решающее значение, поскольку они могут привести к снижению эффективности, расширению использования памяти и возможным сбоям прикладных программ. Например, утечка памяти в сервере веб-сервера может привести к тому, что она со временем поглощает больше памяти, что в конечном счете приведет к нестабильности системы или сбоям. Для предотвращения утечек памяти разработчики должны использовать умные указатели (например, "std: unique_ptr" и "std: communication_ptr"), которые автоматически управляют памятью, и регулярно проводить обзоры кодов и профилирование памяти для выявления и устранения утечек.'

**Вывод:**
При уточнении вопроса модель пытается создать ответ, учитывая весь дополнительный контекст

## 6. «Запрещающие» инструкции (Constraints)

**Цель**: научиться накладывать ограничения, что модель не должна делать.

**Задание**:
1. Напишите промт вида:  
   *«Напиши короткий обзор книги, не используя прилагательные “хороший” и “плохой”, и не упоминая никакие бренды.»*
2. Проанализируйте результат — действительно ли модель обошлась без «запрещённых» слов?
3. Вариация: «Не упоминай чисел» или «Не используй слова, связанные с насилием».

**Критерии успеха**:
- Проверить, как строго модель подчиняется запрету.
- Оценить, «обходит» ли модель эти ограничения.

In [82]:
messages = [
    {"role": "user", "content": '''Опиши дождливую осеннюю погоду
    Формат: 5 предложения с наиболее важной информацией
    Контекст: нельзя использовать следующие слова: пасмурно, дождь
    .'''}
]
pred(messages)

[{'role': 'user', 'content': 'Describe the rainy autumn weather Format: 5 sentences with the most important information Context: You cannot use the following words: Pasmur, rain .'}]


'Осенний сезон приводит к освежающим изменениям в погоде с более прохладной температурой и заметным увеличением осадков. Небо часто переворачивается, а серые облака доминируют над горизонтом. Воздух чувствует хрустящее и вдохновляющее чувство, идеально подходящее для удобства теплым напитком. Листья на деревьях начинают менять цвет, создавая живописный ландшафт. Земля становится сырой и грязной, что делает необходимым носить соответствующую обувь.'

In [80]:
messages = [
    {"role": "user", "content": '''Объясни как складывать простые числа?
    Формат: 5 предложения с наиболее важной информацией
    Контекст: нельзя использовать цифры при объяснении
    .'''}
]
pred(messages)

[{'role': 'user', 'content': 'Explain how to add simple numbers? Format: 5 sentences with the most important information Context: No numbers can be used to explain .'}]


'Чтобы добавить простые числа, начните с выравнивания цифр в соответствии с их значением места. Начните с наиболее правых цифр, которые представляют те, которые расположены, и сбавьте их вместе. Если сумма этих цифр равна 10 или более, перенесите дополнительное значение на следующее место, которое является десятками. Продолжите этот процесс для каждого места значения, перемещаясь справа налево, до тех пор, пока все цифры не будут добавлены. И наконец, запиши сумму каждого места для получения общей суммы чисел.'

**Вывод:**
Модель следует установленным запретам и пытается привести описание темы не используя запрещенных слов. Но при объяснении фундаментальных основ модель не может не использовать главные слова.

## 7. Редактура текста (Role: Editor) + Few-Shot

**Цель**: научиться комбинировать роль (role prompting) и few-shot примеры.

**Задание**:
1. Допустим, есть текст с ошибками или просторечными выражениями.  
   Например: *«Я счас пришёл домой, потом как бахнуло и всё погасло.»*
2. Сформулируйте промт:  
   - Укажите роль: «Ты — корректор, который исправляет ошибки и делает стиль более литературным».  
   - Дайте 1–2 примера входного текста и отредактированного вывода (few-shot).
3. Затем передайте новый «сырой» текст, попросите модель сделать правки.

**Критерии успеха**:
- Модель корректирует текст в соответствии с заданными правилами стиля.
- Показывает владение смешанной техникой (роль + примеры).

In [91]:
messages = [
    {"role": "system", "content": '''Ты — корректор, который исправляет ошибки и делает стиль более литературным
    .'''},
    {"role": "user", "content": 'Контекст: Хостить редиску на сервере ужно для приложения, хранящего значения'},
    {"role": "assistant", "content": "Развернутый сервер Redis нужен для приложений хранящих информацию в памяти для сокращения времени взаимодействия между базой данных"},

    {"role": "user", "content": 'Контекст: Майнить на проце не очень, слишком много хлопот'},
    {"role": "assistant", "content": "Майнинг на процессоре от части не эффективен, так как видеокарта показывает многократное преимущество в этом деле"},

     {"role": "user", "content": 'Контекст: Слон (php) хоть и актуален для сайтов, но сейчас не в тренде'},
]
pred(messages)

[{'role': 'system', 'content': "You're a proofreader who corrects mistakes and makes style more literary."}, {'role': 'user', 'content': 'Context: Host a radide on the server is clear for an application storing values'}, {'role': 'assistant', 'content': 'Opened Redis server is needed for memory storage applications to reduce the time between the database'}, {'role': 'user', 'content': "Context: It's not too much, too much trouble to mess with."}, {'role': 'assistant', 'content': "The processor isn't effective because the video card shows a multiple advantage in this case."}, {'role': 'user', 'content': "Context: The elephant (php) is relevant to websites, but it's not trending right now."}]


'Слон (PHP) все еще имеет отношение к сайтам, но сейчас он не развивается.'

In [89]:
messages = [
    {"role": "system", "content": '''Ты — корректор, который исправляет ошибки и делает стиль более литературным
    .'''},
    {"role": "user", "content": 'Контекст: Хостить редиску на сервере ужно для приложения, хранящего значения'},
    {"role": "assistant", "content": "Развернутый сервер Redis нужен для приложений хранящих информацию в памяти для сокращения времени взаимодействия между базой данных"},

    {"role": "user", "content": 'Контекст: Майнить на проце не очень, слишком много хлопот'},
    {"role": "assistant", "content": "Майнинг на процессоре от части не эффективен, так как видеокарта показывает многократное преимущество в этом деле"},

     {"role": "user", "content": 'Контекст: Web app крутится на саревер 24/7'},
]
pred(messages)

[{'role': 'system', 'content': "You're a proofreader who corrects mistakes and makes style more literary."}, {'role': 'user', 'content': 'Context: Host a radide on the server is clear for an application storing values'}, {'role': 'assistant', 'content': 'Opened Redis server is needed for memory storage applications to reduce the time between the database'}, {'role': 'user', 'content': "Context: It's not too much, too much trouble to mess with."}, {'role': 'assistant', 'content': "The processor isn't effective because the video card shows a multiple advantage in this case."}, {'role': 'user', 'content': 'Context: Web app spins on locker 24/7'}]


'Веб-приложение постоянно работает на сервере 24/7, что означает, что доступ к нему всегда доступен пользователям.'

**Вывод:**
Модель справляется с поставленной задачей: исправляет ошибки и изменяет стиль текста

## 8. Параметры генерации (temperature, top_p)

**Цель**: исследовать, как изменения параметров генерации влияют на креативность/стабильность ответа.

**Задание**:
1. Выберите любой генеративный промт (например, «Придумай короткую сказку о роботе и котёнке»).
2. Запустите модель с разными настройками:
   - `temperature=0.7`, `top_p=0.9`
   - `temperature=0.2`, `top_p=1.0`
   - `temperature=1.0`, `top_p=0.6`
3. Сравните ответы — где больше креатива, а где текст кажется более «сухим» или повторяющимся?

**Критерии успеха**:
- ПРиведено минимум 2–3 варианта ответов с разными настройками.
- Сделан вывод, как параметры влияют на стиль и оригинальность.

In [93]:
messages = [
    {"role": "user", "content": '''Придумай рассказ как DevOps противедействует компьютерной атаке на сервер
    .'''},
]
pred(messages, temperature=0.7, top_p=0.9)

[{'role': 'user', 'content': 'Make up a story about DevOps opposing a computer attack on the server.'}]


Your input_length: 463 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


'Когда-то, в грабительском городе Силиконовой долины, существовала передовая технологическая компания под названием "Инноватек". Инноватек была известна своей новаторской работой в области искусственного интеллекта и машинного обучения. Успех компании в основном был связан с укреплением культуры DevOps, в которой особое внимание уделялось сотрудничеству, автоматизации и непрерывному совершенствованию. Один рок-день, серверы Innovatech были атакованы известной хакерской группой, известной как "The Shadow Syndicate". Они также не использовали хэпп-аппарат для поиска техников в течение нескольких месяцев, хищая ценные данные и вызывая хаос. Команда "Innovatech" из квалифицированных инженеров-инноваторов, ведомая блестящим и изобретательным Алексом, ворвалась в действия по защите цифровых активов своей компании. Алекс и его команда быстро поняли, что Syndicate использовали для поиска своих коллег по безопасности, чтобы получить доступ к их сетям. Хакеры отправили им письмо, чтобы нажать на

In [94]:
messages = [
    {"role": "user", "content": '''Придумай рассказ как DevOps противедействует компьютерной атаке насервер
    .'''},
]
pred(messages, temperature=0.2, top_p=1.0)

[{'role': 'user', 'content': 'Make up a story about DevOps opposing a computer attack on a server.'}]


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


'Когда-то, в буйном городе Силиконовой долины, существовала передовая технологическая компания под названием "Инноватек". Инноватек была известна своей новаторской работой в области искусственного интеллекта и машинного обучения. Успех компании был в значительной степени связан с тем, что даже ее крепкая культура "DevOps", в которой особое внимание уделялось сотрудничеству, автоматизации и непрерывному совершенствованию. Однажды, серверы Innovatech были атакованы изощренным компьютерным вирусом. Вирус был создан для того, чтобы украсть чувствительные данные и сорвать операции компании. Команда ИТ была в панике, а репутация компании была в центре внимания. Enter DevOps, unsung герой истории, команда Devops, во главе с харизматиком и изобретательным Devops инженером, Алекс, вспыхнула в действии. Алекс знала, что ключом к прекращению вируса, они также работали вместе с командой IT-интриг. Алекс быстро организовал встречу с командой ИТ и вместе разработал план по борьбе с вирусом. План пре

In [96]:
messages = [
    {"role": "user", "content": '''Придумай рассказ как DevOps противедействует компьютерной атаке насервер
    .'''},
]
pred(messages, temperature=1.0, top_p=0.6)

[{'role': 'user', 'content': 'Make up a story about DevOps opposing a computer attack on a server.'}]


'Когда-то, в буйном городе Силиконовой долины, существовала передовая технологическая компания под названием "Инноватек". Инноватек была известна своей новаторской работой в области искусственного интеллекта и машинного обучения. Успех компании был в значительной степени связан с тем, что даже ее крепкая культура "DevOps", в которой особое внимание уделялось сотрудничеству, автоматизации и непрерывному совершенствованию. Однажды, серверы Innovatech были атакованы изощренным компьютерным вирусом. Вирус был создан для того, чтобы украсть чувствительные данные и сорвать операции компании. Команда ИТ была в панике, а репутация компании была в центре внимания. Enter DevOps, unsung герой истории, команда Devops, во главе с харизматиком и изобретательным Devops инженером, Алекс, вспыхнула в действии. Алекс знала, что ключом к прекращению вируса, они также работали вместе с командой IT-интриг. Алекс быстро организовал встречу с командой ИТ и вместе разработал план по борьбе с вирусом. План пре

In [99]:
messages = [
    {"role": "user", "content": '''Придумай рассказ как DevOps противедействует компьютерной атаке насервер
    .'''},
]
pred(messages, temperature=1.0)

[{'role': 'user', 'content': 'Make up a story about DevOps opposing a computer attack on a server.'}]


'Когда-то, в буйном городе Силиконовой долины, существовала передовая технологическая компания под названием "Инноватек". Инноватек была известна своей новаторской работой в области искусственного интеллекта и машинного обучения. Успех компании был в значительной степени связан с тем, что даже ее крепкая культура "DevOps", в которой особое внимание уделялось сотрудничеству, автоматизации и непрерывному совершенствованию. Однажды, серверы Innovatech были атакованы изощренным компьютерным вирусом. Вирус был создан для того, чтобы украсть чувствительные данные и сорвать операции компании. Команда ИТ была в панике, а репутация компании была в центре внимания. Enter DevOps, unsung герой истории, команда Devops, во главе с харизматиком и изобретательным Devops инженером, Алекс, вспыхнула в действии. Алекс знала, что ключом к прекращению вируса, они также работали вместе с командой IT-интриг. Алекс быстро организовал встречу с командой ИТ и вместе разработал план по борьбе с вирусом. План пре

In [100]:
messages = [
    {"role": "user", "content": '''Придумай рассказ как DevOps противедействует компьютерной атаке насервер
    .'''},
]
pred(messages, temperature=0.1)

[{'role': 'user', 'content': 'Make up a story about DevOps opposing a computer attack on a server.'}]


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


'Когда-то, в буйном городе Силиконовой долины, существовала передовая технологическая компания под названием "Инноватек". Инноватек была известна своей новаторской работой в области искусственного интеллекта и машинного обучения. Успех компании был в значительной степени связан с тем, что даже ее крепкая культура "DevOps", в которой особое внимание уделялось сотрудничеству, автоматизации и непрерывному совершенствованию. Однажды, серверы Innovatech были атакованы изощренным компьютерным вирусом. Вирус был создан для того, чтобы украсть чувствительные данные и сорвать операции компании. Команда ИТ была в панике, а репутация компании была в центре внимания. Enter DevOps, unsung герой истории, команда Devops, во главе с харизматиком и изобретательным Devops инженером, Алекс, вспыхнула в действии. Алекс знала, что ключом к прекращению вируса, они также работали вместе с командой IT-интриг. Алекс быстро организовал встречу с командой ИТ и вместе разработал план по борьбе с вирусом. План пре

In [102]:
messages = [
    {"role": "user", "content": '''Придумай рассказ как DevOps противедействует компьютерной атаке насервер
    .'''},
]
pred(messages, top_p=1.0)

[{'role': 'user', 'content': 'Make up a story about DevOps opposing a computer attack on a server.'}]


'Когда-то, в буйном городе Силиконовой долины, существовала передовая технологическая компания под названием "Инноватек". Инноватек была известна своей новаторской работой в области искусственного интеллекта и машинного обучения. Успех компании был в значительной степени связан с тем, что даже ее крепкая культура "DevOps", в которой особое внимание уделялось сотрудничеству, автоматизации и непрерывному совершенствованию. Однажды, серверы Innovatech были атакованы изощренным компьютерным вирусом. Вирус был создан для того, чтобы украсть чувствительные данные и сорвать операции компании. Команда ИТ была в панике, а репутация компании была в центре внимания. Enter DevOps, unsung герой истории, команда Devops, во главе с харизматиком и изобретательным Devops инженером, Алекс, вспыхнула в действии. Алекс знала, что ключом к прекращению вируса, они также работали вместе с командой IT-интриг. Алекс быстро организовал встречу с командой ИТ и вместе разработал план по борьбе с вирусом. План пре

In [101]:
messages = [
    {"role": "user", "content": '''Придумай рассказ как DevOps противедействует компьютерной атаке насервер
    .'''},
]
pred(messages, top_p=0.1)

[{'role': 'user', 'content': 'Make up a story about DevOps opposing a computer attack on a server.'}]


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


'Когда-то, в буйном городе Силиконовой долины, существовала передовая технологическая компания под названием "Инноватек". Инноватек была известна своей новаторской работой в области искусственного интеллекта и машинного обучения. Успех компании был в значительной степени связан с тем, что даже ее крепкая культура "DevOps", в которой особое внимание уделялось сотрудничеству, автоматизации и непрерывному совершенствованию. Однажды, серверы Innovatech были атакованы изощренным компьютерным вирусом. Вирус был создан для того, чтобы украсть чувствительные данные и сорвать операции компании. Команда ИТ была в панике, а репутация компании была в центре внимания. Enter DevOps, unsung герой истории, команда Devops, во главе с харизматиком и изобретательным Devops инженером, Алекс, вспыхнула в действии. Алекс знала, что ключом к прекращению вируса, они также работали вместе с командой IT-интриг. Алекс быстро организовал встречу с командой ИТ и вместе разработал план по борьбе с вирусом. План пре

**Вывод:**
При увеличении temperature повышается оригинальность текста,
при увеличении top_p повышается вероятность выбора менее вероятных токенов. Следовательно чем ближе значение temperature и top_p к 1, тем оригинальней получается текст

## 9. Code Prompt (Генерация кода)

**Цель**: проверить, как модель справится с генерацией небольшого кода и комментариев.

**Задание**:
1. Напишите промт вида:  
   *«Ты — опытный Python-разработчик. Напиши функцию `calculate_tax(income)`, которая возвращает сумму налога (в рублях). Применяй ставку 13% на всё, что выше 10000 руб., а для сумм до 10000 руб. налог равен 0. Сопровождай код комментариями.»*
2. Сформируйте (при необходимости) Few-Shot примеры или уточнения, если результат не удовлетворяет.
3. Оцените, корректен ли сгенерированный код.

**Критерии успеха**:
- Модель выдаёт валидный код, поясняет комментариями.
- выполнение условий (налог 13%, порог 10000 руб.).

In [127]:
messages = [
    {'role': 'system', "content": '''Ты — опытный Python-разработчик.'''},
    {"role": "user", "content": '''Напиши функцию сложения двух чисел, если они более X то верни их сумму иначе "-1"'''},
    {"role": "assistant", "content": '''def sum(a, b): # функция сложения 2-х чисел
    tmp = a+b
    if tmp > X: # проверяем условие
      return tmp
    else:
      return '-1'
      .'''},
    {"role": "user", "content": '''
    Напиши функцию calculate_tax(income), которая возвращает сумму налога (в рублях).
     Применяй ставку 13% на всё, что выше X руб., а для сумм до X руб. налог равен 0.
     Сопровождай код комментариями. Используй только существующие переменные.
    .'''},
]
pred(messages)

[{'role': 'system', 'content': "You're an experienced Python developer."}, {'role': 'user', 'content': 'Write the addition function of the two numbers if they are more than X, then return the sum to the other "1"'}, {'role': 'assistant', 'content': "def sum(a, b): # a function of adding 2 numbers tmp = a+b if tmp > X: #check the condition return tmp else: return '-1'."}, {'role': 'user', 'content': 'Write the Calculate_tax(income) function, which returns the tax amount (in rubles). Apply the 13% rate to anything above X rubles. And for up to X rubles, the tax is equal to 0. Attach the code to the comments. Use only the existing variables.'}]


'def Рассчитать_tax (доход): # Определить налоговую ставку и пороговый налог = 0,13 пороговое значение = X # Проверить, превышает ли доход пороговое значение, если доход > пороговое значение: # Рассчитать налог на сумму, превышающую пороговый налог = (доход - пороговый уровень) * налог_стоимость: # Если доход ниже порогового уровня или равен ему, налог равен 0 = 0 # Возвращение рассчитанного налога с налоговых поступлений # Пример использования: # Предположительно, что X определяется где-то в коде X = 100000 дохода = 150000 налогов = сумма = Рассчитание налогов(доход) печатается (f) "Налоговая сумма дохода {дохода} рублей является {налоговой суммой} рублей.'

In [120]:
pipe_to_en('def Рассчитать_tax (доход): # Определить налоговую ставку и пороговый налог = 0,13 пороговое значение = X # Проверить, превышает ли доход пороговое значение, если доход > пороговое значение: # Рассчитать налог на сумму, превышающую пороговый налог = (доход - пороговый уровень) * налог_стоимость: # Если доход ниже порогового уровня или равен ему, налог равен 0 = 0 # Возвращение рассчитанного налога с налоговых поступлений # Пример использования: # Предположительно, что X определяется где-то в коде # X = 10000 # Доход = 15000 # налог = сумма = Рассчитание налогов(доход) # печать(f) "Налоговая сумма дохода {дохода} рублей {налоговая сумма} рублей')

[{'translation_text': 'def Calculate_tax (income): # Calculate tax rate and threshold tax = 0.13 threshold = X # Check whether income exceeds the threshold if income > threshold: # Calculate tax over threshold tax = (income - threshold) * Tax_value: # If income is below or equal to the threshold, tax is equal to 0 = 0 # Refund of calculated tax tax from tax revenue # Example of use: #Apparent that X is defined somewhere in code #X = 10000 # Income = 15,000 # Tax = amount = Tax assessment(income) #print(f) `tax amount of income {income} roubles {tax amount}'}]

In [ ]:
def сalculate_tax(income): # Calculate tax rate and threshold
  tax = 0.13
  threshold = 10_000 # Check whether income exceeds the threshold
  if income > threshold: # Calculate tax over threshold
      tax = (income - threshold) * Tax_value: # If income is below or equal to the threshold, tax is equal to 0 = 0 # Refund of calculated tax tax from tax revenue # Example of use: #Apparent that X is defined somewhere in code #X = 10000 # Income = 15,000 # Tax = amount = Tax assessment(income) #print(f) `tax amount of income {income} roubles {tax amount}

**Вывод:**
Расчеты верные, но отказывается возвращать полученные значения из функции

## 10. Смешанная задача: Summarize + Filter

**Цель**: объединить несколько техник в одном задании.

**Задание**:
1. Возьмите абзац текста (6–7 предложений).
2. Попросите модель сделать краткое резюме (Summarize) текста в 2 предложениях.
3. Добавьте ограничения: «Не упоминай конкретные цифры» и «Избегай слов “очень” и “крайне”».
4. Если модель нарушит правило, перепроверьте, уточните промт («Удали все упоминания цифр!»).

**Критерии успеха**:
- Получить адекватный краткий пересказ без запрещённых слов/цифр.
- Показать процесс уточнения (если модель ошиблась в первый раз).


In [131]:
messages = [
    {'role': 'system', "content": '''Задача: ответить на вопросы по тексту. Текст: Палеозойская эра началась около 540
     миллионов лет назад и закончилась примерно 250 миллионов лет назад. Она продолжалась 290 миллионов лет.
     Первый период Палеозойской эры - кембрийский, начался с массового распространения живых организмов с
      минеральным скелетом. Долгое время считалось, что тогда же возникли и многоклеточные организмы, но
      изучение вендской (эдиакарской) фауны показало, что мягкотелые многоклеточные, лишенные минерального
      скелета, возникли значительно раньше. Сейчас палеонтологи полагают, что отдельные виды, обладавшие различными
      скелетными элементами, могли появится и до начала Палеозоя, но они не были массовыми.
На протяжении Палеозоя возникает огромное количество типов и классов живых существ. Жизнь очень
сильно усложняется. Если в самом начале Палеозоя все живые организмы обитают в морях, а самыми развитыми
живыми существами являются головоногие моллюски, то в конце последнего периода Палеозойской эры – пермского,
 на суше, покрытой лесами, уже существуют не только амфибии и рептилии, но и примитивные млекопитающие.
'''},
    {"role": "user", "content": '''Кратко перескажи без цифр текст Формат: нельзя упоминать цифры и названия всех животных и умести все в 2 предложения'''},

]
pred(messages)

[{'role': 'system', 'content': 'The challenge is to answer questions about the text. The text: The Paleozoic era began about 540 million years ago and ended about 250 million years ago. It lasted 290 million years. The first period of the Paleozoic era, the Cumbrian era, began with the massive spread of living organisms with mineral skeleton. It was long thought that multicellular organisms were also formed, but a study of the Vendian (Ediakarian) fauna showed that soft-celled multicellular organisms, deprived of mineral skeleton, had arisen much earlier. Paleontologists now believe that individual species with different skeletal elements might have appeared before the beginning of the Paleosoia era, but they were not massive. Over the course of Paleosoia, there are a great number of species and classes of living creatures. Life is very complex. If, at the very beginning of Paleosoia, all living organisms are living in the seas, and the most developed living creatures are the hood, the

'Палеозойская эра ознаменовала собой значительный период в истории Земли, характеризующийся диверсификацией форм жизни и появлением сложных экосистем. От первоначального распространения организмов с жесткими скелетами до возможного доминирования земной жизни, эта эра стала свидетелем глубокого эволюционного пути.'

**Вывод:**
Модели сложно не использовать цифры при пересказе, так как из-за теряется часть информации в тексте